# TRABAJO Parte 2: AIA_2022-2023

# Nombre y DNI del alumno/a 1: Santiago Miñarro García


# Transfer Learning con CNNs - Dataset: Flowers

La idea de este trabajo es familiarizarnos con dos situaciones muy habituales en la actividad real de un "Machine Learning Engineer":

1.   En primer lugar, con una de las técnicas más potentes asociadas con las redes neuronales: el **Transfer Learning**. Dado que las redes neuronales, para resolver un problema, capturan en su estructura de capas y pesos una representación jerárquica del problema.
Entonces..., ¿por que no aprovechar ese conocimiento obtenido, para resolver otro problema diferente?

2.  En segundo lugar, con la **busqueda de información sobre conceptos nuevos**. En este caso, los dos primeros modelos a implementar los hemos trabajado en clase. No así el Transfer Learning, y por tanto, debereis buscar vosotros mismos como hacer lo que se pide para el Modelo 3. Consultar en blogs, web y tutoriales es algo común en el día a día de alguien que quiere profundizar en el ML y, para ello, existen infinidad de fuentes. A modo de ejemplo, una fuente para profundizar en el Transfer Learning con redes convolucionales es: https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/

En este trabajo vamos intentar resolver un problema de clasificación sobre un dataset propuesto por Tensorflow en 2019 conocido como "flowers". Este conjunto está formado por 3670 imágenes de flores pertenecientes a 5 clases diferentes. Para ello implementaremos 3 modelos:

*   Modelo 1: implementación de una CNN básica.
*   Modelo 2: es una evolución del modelo anterior, aplicando técnicas que reduzcan el overfitting.
*   Modelo 3: rompemos la barrera de tener que seguir complicando nuestro modelo y se pide aplicar transfer learning utilizando un pre-trained model.





# a) Carga de datos

In [3]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import pandas as pd

import pathlib
import os
import glob
import shutil
tf.__version__

'2.15.0'

Descargamos el dataset que pone a nuestra disposición Tensorflow.

In [4]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL,
                                   fname="flower_photos.tgz",
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228813984/228813984 [==============================] - 2s 0us/step


Tras completar la descarga, debemos tener la siguiente estructura de directorios:  

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower_photos</b>
|__ <b>daisy</b>
|__ <b>dandelion</b>
|__ <b>roses</b>
|__ <b>sunflowers</b>
|__ <b>tulips</b>
</pre>

Desgraciadamente, para este dataset, Tensorflow no nos proporciona la estructura de directorios necesaria de train y de validación. Por lo que debemos proceder del siguiente modo:

* Crear una carpeta `train` y de `val`, cada una de ellas debe contener a su vez, cinco subdirectorios: uno para cada clase de flor.
* Moveremos las imágenes de las carpetas originales a estas nuevas carpetas. De modo que el 80% de las imágenes vayan al conjunto de train y el 20% restante al de validación.
* La estructura final de directorios debe ser la siguiente:

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower_photos</b>
|__ <b>train</b>
    |______ <b>daisy</b>: [12.jpg, 28.jpg, 31.jpg ....]
    |______ <b>dandelion</b>: [41.jpg, 22.jpg, 35.jpg ....]
    |______ <b>roses</b>: [121.jpg, 92.jpg, 38.jpg ....]
    |______ <b>sunflowers</b>: [93.jpg, 23.jpg, 83.jpg ....]
    |______ <b>tulips</b>: [109.jpg, 267.jpg, 93.jpg ....]
 |__ <b>val</b>
    |______ <b>daisy</b>: [507.jpg, 508.jpg, 509.jpg ....]
    |______ <b>dandelion</b>: [719.jpg, 720.jpg, 721.jpg ....]
    |______ <b>roses</b>: [514.jpg, 515.jpg, 516.jpg ....]
    |______ <b>sunflowers</b>: [560.jpg, 561.jpg, 562.jpg .....]
    |______ <b>tulips</b>: [640.jpg, 641.jpg, 642.jpg ....]
</pre>

Creamos una lista con el nombre de las 5 clases. En castellano sería: margaritas, diente de león, rosas, girasoles y tulipanes.

In [6]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

Creemos la estructura de directorios necesaria:

In [7]:
SPLIT_RATIO=0.8

for cl in classes:
    # path de las imagenes de la clase cl
    img_path = os.path.join(base_dir, cl)

    # obtenemos la lista de todas las imagenes
    images = glob.glob(img_path + '/*.jpg')
    print("{}: {} Imagenes".format(cl, len(images)))

    # determinamos cuantas imagenes son el 80%
    num_train = int(round(len(images)*SPLIT_RATIO))

    # separamos las imagenes en dos listas
    train, val = images[:num_train], images[num_train:]

    # creamos la carpeta de train/clase y val/clase
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
        os.makedirs(os.path.join(base_dir, 'train', cl))
    else:
        shutil.rmtree(os.path.join(base_dir, 'train', cl))

    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
        os.makedirs(os.path.join(base_dir, 'val', cl))
    else:
        shutil.rmtree(os.path.join(base_dir, 'val', cl))

    for t in train:
        shutil.move(t, os.path.join(base_dir, 'train', cl))

    for v in val:
        shutil.move(v, os.path.join(base_dir, 'val', cl))

roses: 641 Imagenes
daisy: 633 Imagenes
dandelion: 898 Imagenes
sunflowers: 699 Imagenes
tulips: 799 Imagenes


Preparamos variables con las rutas de los diferentes directorios:

In [8]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

daisy_dir = os.path.join(train_dir, 'daisy')
dandelion_dir = os.path.join(train_dir, 'dandelion')
roses_dir = os.path.join(train_dir, 'roses')
sunflowers_dir = os.path.join(train_dir, 'sunflowers')
tulips_dir = os.path.join(train_dir, 'tulips')

**Tarea 1: Muestre el nombre de dos ficheros cualquiera en alguna de esas rutas**

In [ ]:
# El código aquí
archivos_daisy = os.listdir(daisy_dir)
print(archivos_daisy[0],archivos_daisy[1])
print(daisy_dir)

NotADirectoryError: [Errno 20] Not a directory: '/root/.keras/datasets/flower_photos/train/daisy'

Es decir, la clase a la que pertenece cada imagen no viene dada por el nombre del fichero sino por el directorio en el que se encuentra almacenada.

**Tarea 2: Muestra el número de imágenes de train que tenemos de cada clase**

In [ ]:
# El código aquí
for i in os.listdir(train_dir):
  print("Clase " + i + ": " + str(len(os.listdir(train_dir + "/" + i ))))

# b) Visualización del dataset

**Tarea 3: Muestra 3 imágenes de cada una de las clases, el título de la imagen será el shape del array de numpy asociado a la imagen**

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(16,12))
subplot_contador = 1

for categoria in os.listdir(train_dir):
    dir_categoría = os.path.join(train_dir, categoria)
    archivos = os.listdir(dir_categoría)

    for j in range(3):
        if j < len(archivos):
            ruta_imagen = os.path.join(dir_categoría, archivos[j])
            imagen = mpimg.imread(ruta_imagen)
            ax = plt.subplot(len(os.listdir(train_dir)), 3, subplot_contador)
            ax.axis('off')
            ax.imshow(imagen)
            ax.set_title(f"{categoria} - {j+1}")
            subplot_contador += 1


plt.show()


# c) Modelo 1: CNN básica (objetivo: accuracy_valid > 60%)


Implemente una red convolucional para resolver el problema de clasificación. Para ello se sugiere una CNN con 3 capas convolucionales + pooling con la siguiente estructura:

Bloque de procesamiento de imagen:
1.   32 kernels -> 64 kernels -> 96 kernels
2.   kernels de 3x3.
3.   Stride = 1 y padding = SI.
4.   Función de activación ReLU.
5.   Maxpooling de 2x2 con stride clásico de 2 pixeles.
6.   Igualamos el tamaño de todas las imágenes a 150 x 150.

Bloque de decisión:
7.   Capa densa de 512 neuronas.
8.   Capa densa de salida.

**Tarea 4: Define un modelo con la estructura anterior**



In [ ]:
# El código aquí
model = keras.Sequential()
model.add(keras.layers.Conv2D(
              filters=32,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_1',
              input_shape=(150, 150, 3)))


model.add(keras.layers.MaxPooling2D(
              pool_size=(2, 2),
              name='Pool_2',
              strides=(2, 2)))

model.add(keras.layers.Conv2D(
              filters=64,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_2',
              input_shape=(75, 75, 32)))

model.add(keras.layers.MaxPooling2D(
              pool_size=(2, 2),
              name='Pool_1',
              strides=(2, 2)))

model.add(keras.layers.Conv2D(
              filters=64,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_3',
              input_shape=(37, 37, 64)))


model.add(tf.keras.layers.Flatten())


model.add(keras.layers.Dense(units=512,name='densa_1', activation='relu'))
model.add(keras.layers.Dense(units=1, activation='softmax'))

**Tarea 5: Indica el shape de la imagen antes y después de cada capa de la red. Explica cómo has obtenido dichos valores**

|Capa| Shape a la salida| #parámetros |
|:-|:-:|:-:|
|Conv_1|150 x 150 x 32|
|Pool_1| 75 x 75 x 32|
|Conv_2| 75 x 75 x 64|
|Pool_2| 37 x 37 x 64|
|flatten| 87616 |
|densa_1| 1 |






Transformación de Dimensiones en una Red Neuronal Convolucional

Primera Capa - Conv_1:

Input Shape: 150×150×3

Esta es la forma de las imágenes ingresadas: 150x150 píxeles con 3 canales de color (RGB).

Parámetros de la Capa:

Filtros: 32

Tamaño del Kernel:
3
×
3

Stride:
(1
,
1)

Padding: 'same'

Output Shape: Debido al padding 'same' y al stride de (1,1), la dimensión espacial de la salida permanece igual pero el número de canales cambia al número de filtros. Por lo tanto, el shape de salida es
150
×
150
×
32

Segunda Capa - Pool_1:

Input Shape:
150
×
150
×
32

Parámetros de la Capa:

Tamaño del Pool:
2
×
2

Stride:
(2
,
2)

Output Shape: El pooling reduce cada dimensión espacial a la mitad cuando el tamaño del pool es 2 y el stride es 2. Por lo tanto, el shape de salida es
75
×
75
×
32

Tercera Capa - Conv_2:

Input Shape:
75
×
75
×
32

Parámetros de la Capa:

Filtros: 64

Tamaño del Kernel:
3
×
3


Stride:
(1
,
1)

Padding: 'same'

Output Shape: Con los mismos parámetros de padding y stride, el tamaño espacial se mantiene, cambiando solo el número de canales a 64. El shape de salida es
75
×
75
×
64

Cuarta Capa - Pool_2:

Input Shape:
75
×
75
×
64

Parámetros de la Capa:

Tamaño del Pool:
2
×
2


Stride:
(2
,
2)

Output Shape: Similar al primer MaxPooling, el tamaño espacial se reduce a la mitad. Por lo tanto, el shape de salida es
37
×
37
×
64

Quinta Capa - Conv_3:

Input Shape:
37
×
37
×
64

Parámetros de la Capa:

Filtros: 64

Tamaño del Kernel:
3
×
3

Stride:
(1
,
1)

Padding: 'same'

Output Shape: Manteniendo el padding 'same', el tamaño espacial no cambia y el número de canales sigue siendo 64. El shape de salida es

37 × 37 × 64

**Tarea 6: Compara el resultado con un summary() del modelo**


In [ ]:
# el código aquí
model.summary()

**Tarea 7: Entrena el modelo de manera que obtenga un accuracy (sobre el conjunto de validación) > 60%.**

* Utilice el optimizador que considere más adecuado.
*   Recuerda que si no se realiza conversión a One-Hot de la etiqueta a predecir, debes utilizar como función de error `SparseCategoricalCrossentropy` (este es el procedimiento que hemos usado en clase).
*   Considera un learning rate en el entorno de 0.001.
*   En el caso de los generators utiliza `class_mode='sparse'`.
*   Puedes utilizar p.e. un `batch_size = 100`.

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=100,
        class_mode='sparse')


In [13]:
# El código aquí
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy'])

es_callback = keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)



In [ ]:
history = model.fit(train_generator,
                    batch_size=20,
                    epochs=15,
                    validation_data=validation_generator,
                    callbacks=[es_callback])

**Tarea 8: Muestra la evolución de la función de error (train y valid) durante el entrenamiento.Explica que problema presenta el modelo que hemos entrenado.**

In [ ]:
# El código aquí
pd.DataFrame({'loss_train': history.history['loss'],
              'loss_valid': history.history['val_loss']}).plot(figsize=(8,4))

plt.grid(True)
plt.ylim(0,1)
plt.xlabel('numero de epocas')
plt.title('Training and validation loss');

# d) Modelo 2: reducción del overfitting (objetivo: accuracy_valid > 70%)

Para mejorar el accuracy del modelo, vamos a incorporar las dos técnicas más habituales de reducción del overfitting:

    * drop-out
    * data augmentation
    

**Tarea 9: Explica en que consisten y qué utilidad tienen para nuestro problema**





##### La explicación aqui

Drop-out es la técnica que consiste en "apagar" un porcentaje de neuronas lo que permite esencialmente que la red sea menos propensa a memorizar detalles pequeños y específicos de las imágenes con las que se entrena. Esto ayuda a que la red sea más flexible y mejor en reconocer imágenes nuevas y diferentes, no solo las que ya ha visto durante el entrenamiento.

Data Augmentation consiste en aumentar el número de ejemplos sobre el que tu modelo aprende modificando ligeramente los datos de entrenamiento para crear nuevos de los cuales el modelo pueda aprender. Esto permite que el modelo aprenda a reconocer flores desde diferentes perspectivas y condiciones, aumentando su capacidad de generalizar bien para cuando se enfrente a una nueva imagen que no se encuentra en el conjunto de entrenamiento.

**Tarea 10: Construye un nuevo modelo 2 incorporando (en el modelo 1 anterior) el dropout adecuado**

In [ ]:
model2 = keras.Sequential()
model2.add(keras.layers.Conv2D(
              filters=32,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_1',
              input_shape=(150, 150, 3)))


model2.add(keras.layers.MaxPooling2D(
              pool_size=(2, 2),
              name='Pool_2',
              strides=(2, 2)))

model2.add(keras.layers.Conv2D(
              filters=64,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_2',
              input_shape=(75, 75, 32)))

model2.add(keras.layers.MaxPooling2D(
              pool_size=(2, 2),
              name='Pool_1',
              strides=(2, 2)))

model2.add(keras.layers.Conv2D(
              filters=64,
              kernel_size=(3, 3),
              strides=(1, 1),
              padding='same',
              activation='relu',
              name='conv_3',
              input_shape=(37, 37, 64)))


model2.add(tf.keras.layers.Flatten())


model2.add(keras.layers.Dense(units=512,name='densa_1', activation='relu'))
model2.add(keras.layers.Dropout(0.5))  #Dropout
model2.add(keras.layers.Dense(units=1, activation='softmax'))

**Tarea 11: Explica que tipos de augmentation vas a considerar y que utilidad tienen en nuestro problema de clasificacion**

##### La explicación aqui

Vamos a considerar los siguientes tipos:

Rotaciones -> Añadiremos leves rotaciones a las imágenes para que reflejen una posible realidad en la que la foto esta inclinada hacia la izquierda o hacia la derecha.

Desplazamientos -> Mover las imágenes horizontal o verticalmente para que el modelo pueda reconocer fotos en las que las flores están mal centradas.

Reflejo: Invertir las imágenes horizontalmente para aumentar el número de posibilidades.

Zoom -> Añadiremos zoom para reflejar la situación real en la que la flor se vea de mas cerca.


In [ ]:
train_datagen1 = ImageDataGenerator(
#    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

val_datagen1 = ImageDataGenerator(rescale=1./255)

**Tarea 12: Entrena el modelo de manera que obtenga un accuracy (sobre el conjunto de validación) > 70%.**

In [ ]:
# El código aquí
train_generator1 = train_datagen1.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='sparse')

validation_generator1 = val_datagen1.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='sparse')

model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['acc'])

es_callback = keras.callbacks.EarlyStopping(patience=4, restore_best_weights=True)


history2 = model2.fit(train_generator1,
                    batch_size=20,
                    epochs=15,
                    validation_data=validation_generator1,
                    callbacks=[es_callback])

**Tarea 13: Muestra la evolucion de la funcion de error durante el entrenamiento. Explica qué diferencias de comportamiento hay entre las gráficas del modelo 1 y el modelo 2**

In [ ]:
# El código aquí
pd.DataFrame({'loss_train': history2.history['loss'],
              'loss_valid': history2.history['val_loss']}).plot(figsize=(8,4))

plt.grid(True)
plt.ylim(0,1)
plt.xlabel('numero de epocas')
plt.title('Training and validation loss');

# c) Modelo 3: Transfer Learning (objetivo: accuracy_valid > 88%)

En nuestro problema de clasificación de flores, el utilizar CNNs diseñadas y entrenadas por nosotros mismos, aparece una barrera en las proximidades del 75-77% de precisión del modelo (siempre sobre validación).

Para superar este escollo, el siguiente paso natural es la utilización de modelos preentrenados. Existe una gran variedad de ellos basados en redes CNNs clásicas, donde la principal diferencia es que acumulan más capas que nuestros modelos 1 y 2. Adicionalmente, en estos modelos se han ido incorporando diferentes propuestas para mejorar la arquitectura de la CNN.

En general, utilizando estos modelos convolucionales preentrenados podemos alcanzar accuracies próximos al 90%. Normalmente, estos modelos han sido previamente entrenados sobre datasets de gran tamaño y con gran número de categorías. P.e. en subconjuntos de Imagenet (14 millones de imagenes de 22K categorías).

Dado que estos modelos se entrenaron para resolver un problema "relativamente" parecido a nuestro problema de clasificación, parece razonable pensar que podemos aprovechar ese conocimiento capturado en la red para resolver nuestro problema de clasificación de flores.



Para elegir el modelo preentrenado que debe utilizar cada grupo, proceda del siguiente modo:
* Paso 1: sume los DNIs de los componentes del grupo (si el grupo tiene un sólo miembro, vaya directamente al paso 2). res = dni_1 + dn_2
* Paso 2: Aplique la siguiente operación al resultado anterior: res mod 6.
* Paso 3: Tome el modelo cuyo número asociado coincide con el resultado de la operación anterior.
* Paso 4: El porcentaje que aparece entre paréntesis junto al nombre del modelo es el accuracy (en validación) que deberías poder alcanzar sin dificultad utilizando el modelo. En todos los casos considera un input_shape = (224, 224, 3).

  0. Resnet50 (>90%)
  https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50

  1. Resnet101 (>90%) input_shape = (224, 224, 3)
  https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet/ResNet101

  2. VGG16 (>90%) input_shape = (224, 224, 3)
  https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16

  3. VGG19 (>90%) input_shape = (224, 224, 3) https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg19/VGG19

  4. Xception (>88%) input_shape = (224, 224, 3) https://www.tensorflow.org/api_docs/python/tf/keras/applications/xception/Xception

  5. Inceptionv3 (>88%) input_shape = (224, 224, 3) https://www.tensorflow.org/api_docs/python/tf/keras/applications/inception_v3/InceptionV3

P.e. si los DNIs de los alumnos son: 12345678 y 23456781. La suma es 35802459. De donde 35802459 mod 6 = 3. Por tanto, tomaríamos el modelo VGG16.

Recuerda que puedes utilizar el siguiente post como referencia del uso de transfer learning: https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/

**Tarea 14: Importa el modelo desde Tensorflow**

Dado que este modelo ha sido entrenado para clasificar entre 1.000 categorías, las capas densas finales del modelo no son útiles para nuestro problema de clasificación de 5 categorías (es lo que suele llamarse include_top = SI/NO). De manera que eliminamos lo que a veces se suele llamar el "top model". De este modo sólo nos quedamos con la parte que hace la funcionalidad de "procesamiento" de la imagen.

In [1]:
# El código aquí
from keras.applications.xception import Xception,preprocess_input
from keras.models import Model

**Tarea 15: Personalizar el bloque de decisión**  
Añadimos una capa de flatten y tres nuevas capas densas especificas para nuestro problema con dimensiones 4096, 1072 y la que necesite la capa de salida (con sus correspondientes drop-outs).

In [9]:
# El código aquí

input_shape =(224,224,3)
n_clases = 5


conv_base = Xception(include_top = False,input_shape = input_shape)
top_model = conv_base.output
top_model = keras.layers.Flatten(name="flatten")(top_model)
top_model = keras.layers.Dense(4096, activation='relu')(top_model)
top_model = keras.layers.Dense(1072, activation='relu')(top_model)
top_model = keras.layers.Dropout(0.2)(top_model)
output_layer = keras.layers.Dense(n_clases,activation='softmax')(top_model)





**Tarea 16: Congelar los pesos que no se vayan a entrenar**

Previo a hacer el denominado `Fine-Tuning` del modelo, indicaremos a Tensorflow que únicamente debe entrenar:

   * Las dos últimas capas convolucionales de la red preentrenada, de las que realizaremos un ajuste fino de los pesos.
   * Las tres capas densas que hemos incluido nuevas.

In [10]:
# El código aquí
for layer in conv_base.layers[:-2]:
            layer.trainable = False

**Tarea 17: Crear los datagenerators oportunos**

Para ello:
> * Utiliza Data augmentation.  
> * Las imágenes tienen que ser preprocesadas igual que cuando se entrenó el modelo pre.entrenado original. Para ello se utiliza el parámetro `preprocessing_function=preprocess_input` (preprocess_input importado desde `keras.applications.xxxxxx` en ambos generators (train y valid). Por tanto, no hay que indicarle `rescale`. En caso de ser necesario, se encargará `preprocess_input`.  
> * Dado que estamos reutilizando un modelo que no "es nuestro", deberemos ceñirnos al tamaño de imagen que permite la red a la entrada. Recuerda que debe ser: 224x224.

In [11]:
# El codigo aquí
from keras.preprocessing.image import ImageDataGenerator
train_generator = ImageDataGenerator(rotation_range=40,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    horizontal_flip=True,
                                    validation_split=0.15,
                                    preprocessing_function=preprocess_input)

traingen = train_generator.flow_from_directory(train_dir,
                                               target_size=(224, 224),
                                                    batch_size=100,
                                                    class_mode='sparse')

validgen = train_generator.flow_from_directory(val_dir,
                                               target_size=(224, 224),
                                                batch_size=100,
                                                class_mode='sparse')

Found 2935 images belonging to 5 classes.
Found 735 images belonging to 5 classes.


**Tarea 18: Haz el fine-tuning del modelo con el objetivo de alcanzar un accuracy (sobre el conjunto de validación > 88%).**

A la hora de entrenar un modelo pretrained es típico bajar el learning rate respecto al que utilizaríamos para un modelo nuestro desde cero.

In [14]:
# El código aquí
optim = keras.optimizers.Adam(learning_rate=0.0001)
model = Model(inputs=conv_base.input, outputs=output_layer)
model.compile(optimizer=optim,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

xception_history = model.fit(traingen,
                                  batch_size=100,
                                  epochs=20,
                                  validation_data=validgen,
                                  steps_per_epoch=10,
                                  validation_steps=10,
                                  callbacks=[es_callback])


Epoch 1/20
10/10 [==============================] - ETA: 0s - loss: 1.5549 - accuracy: 0.5970

10/10 [==============================] - 58s 4s/step - loss: 1.5549 - accuracy: 0.5970 - val_loss: 1.1789 - val_accuracy: 0.6912
Epoch 2/20
10/10 [==============================] - ETA: 0s - loss: 0.8132 - accuracy: 0.7519

10/10 [==============================] - 15s 1s/step - loss: 0.8132 - accuracy: 0.7519
Epoch 3/20
10/10 [==============================] - ETA: 0s - loss: 0.5569 - accuracy: 0.8080

10/10 [==============================] - 16s 2s/step - loss: 0.5569 - accuracy: 0.8080
Epoch 4/20
10/10 [==============================] - ETA: 0s - loss: 0.4788 - accuracy: 0.8390

10/10 [==============================] - 16s 2s/step - loss: 0.4788 - accuracy: 0.8390
Epoch 5/20
10/10 [==============================] - ETA: 0s - loss: 0.4113 - accuracy: 0.8470

...

10/10 [==============================] - 15s 2s/step - loss: 0.3302 - accuracy: 0.8824
Epoch 17/20
10/10 [==============================] - ETA: 0s - loss: 0.3570 - accuracy: 0.8710

10/10 [==============================] - 15s 2s/step - loss: 0.3570 - accuracy: 0.8710
Epoch 18/20
10/10 [==============================] - ETA: 0s - loss: 0.3053 - accuracy: 0.8856

10/10 [==============================] - 15s 1s/step - loss: 0.3053 - accuracy: 0.8856
Epoch 19/20
10/10 [==============================] - ETA: 0s - loss: 0.3046 - accuracy: 0.8860

10/10 [==============================] - 16s 2s/step - loss: 0.3046 - accuracy: 0.8860
Epoch 20/20
10/10 [==============================] - ETA: 0s - loss: 0.2356 - accuracy: 0.9180

10/10 [==============================] - 16s 2s/step - loss: 0.2356 - accuracy: 0.9180
